In [1]:
#CNN (Convolutional Neural Network) 
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import LSTM

import  os
from keras.models import Sequential, load_model
from keras.layers.convolutional import Conv1D,MaxPooling1D

from sklearn.preprocessing import MinMaxScaler
import  pandas as pd
import numpy as np

dic={'batch_size': 113.0, 'dropout1': 0.6068791840872675, 'dropout3': 0.5442798896052787, 'dropout4': 0.305422943115805, 'units1': 381.0, 'units3': 67.0, 'units4': 290.0}
forword=30
look_back=120
tr_size=0.86
ep=50

df=pd.read_excel('/kaggle/input/oilprice1/5.xlsx')
df1=df.copy()


df1=df1[['日期（月度）','USA_output','OPEC_output','demand_current','supply_current']]
#df1=df1.bfill()
df1=df1.dropna()


df1=df1.rename(columns={"日期（月度）":'日期（日期）'})
df1=df1.set_index('日期（日期）')


df1=df1.asfreq(freq='d')

df1=df1.bfill()
#print(df1.head(40))

df=df.set_index('日期（日期）')


df=df[['Brent','USD','CRB','Premium','trade','BDI']]


df=df.ffill()
df=df.bfill()



df=df.join(df1)

df=df.dropna()
df=df.astype('float64')
df['Premium']=np.add(df['Premium'],16)
df_raw=df.copy()
df = df.diff().dropna()

#df['flag']=[ (lambda y: 1 if (y%7==6)|(y%7==0) else 0)(x) for x in range(df.shape[0])]
print(df.shape[0])
print(df_raw.shape[0])




#print(df)



#df=df.iloc[0:50]



dataset=df.values
#dataset = dataset.astype('float32')
#print(dataset)
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)






train_size = int(len(dataset) * tr_size)
trainlist = dataset[:train_size]
testlist = dataset[train_size:]



def create_dataset(dataset, look_back,forword):

        dataX, dataY = [], []
        for i in range(len(dataset)-look_back-forword):
            a = dataset[i:(i+look_back)]
            dataX.append(a)
            dataY.append(dataset[(i + look_back):(i+look_back+forword)])
        return np.array(dataX),np.array(dataY)


trainX,trainY  = create_dataset(trainlist,look_back,forword)
testX,testY = create_dataset(testlist,look_back,forword)


trainY=np.reshape(trainY,(trainY.shape[0],-1))

#print(trainX)
#print(trainY)


def trainModel():
    

    model = Sequential()
    model.add(Conv1D(filters=400, kernel_size=2, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2])))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(350, activation='relu'))
    model.add(Dense(trainY.shape[1]))
    model.compile(optimizer='adam', loss='mse')    

    return model



model=trainModel()
model.fit(trainX, trainY, epochs=ep, batch_size=int(dic['batch_size']), verbose=2)



model.save('model2.h5')    



#y_hat  =  model.predict(np.reshape(trainX[0:2],(-1,trainX.shape[1],trainX.shape[2])))
y_hat  =  model.predict(testX)




def retrive(hat):
    hat=np.reshape(hat,(-1,df.shape[1]))
    hat=scaler.inverse_transform(hat)
    hat=np.reshape(hat,(-1,forword,df.shape[1]))
    ind=train_size+look_back-1
    result=[]
    for y in range(hat.shape[0]):
        ind_temp=ind+y
        temp=np.array([ hat[y][x][0] for x in range(forword)])
        las=df_raw.iloc[(ind_temp+1)]['Brent']
        temp=np.insert(temp,0,las)
        temp=np.cumsum(temp)
        result.append(temp[1:])
    #print(np.array(result))
    return np.array(result)


#print(df_raw.iloc[train_size:])

diff=retrive(y_hat)-retrive(testY)


mae=(np.abs(diff)).mean()
#rmse=np.sqrt((diff**2).mean())
print(mae)
fo = open("res.txt", "w")
fo.write(repr(mae))
fo.close()

Using TensorFlow backend.
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

6574
6575
Epoch 1/50
 - 4s - loss: 0.0321
Epoch 2/50
 - 0s - loss: 0.0016
Epoch 3/50
 - 0s - loss: 0.0015
Epoch 4/50
 - 0s - loss: 0.0015
Epoch 5/50
 - 0s - loss: 0.0015
Epoch 6/50
 - 0s - loss: 0.0015
Epoch 7/50
 - 0s - loss: 0.0015
Epoch 8/50
 - 0s - loss: 0.0015
Epoch 9/50
 - 0s - loss: 0.0015
Epoch 10/50
 - 0s - loss: 0.0015
Epoch 11/50
 - 0s - loss: 0.0015
Epoch 12/50
 - 0s - loss: 0.0015
Epoch 13/50
 - 0s - loss: 0.0015
Epoch 14/50
 - 0s - loss: 0.0015
Epoch 15/50
 - 0s - loss: 0.0015
Epoch 16/50
 - 0s - loss: 0.0015
Epoch 17/50
 - 0s - loss: 0.0015
Epoch 18/50
 - 0s - loss: 0.0015
Epoch 19/50
 - 0s - loss: 0.0015
Epoch 20/50
 - 0s - loss: 0.0015
Epoch 21/50
 - 0s - loss: 0.0015
Epoch 22/50
 - 0s - loss: 0.0015
Epoch 23/50
 - 0s - loss: 0.0015
Epoch 24/50
 - 0s - loss: 0.0015
Epoch 25/50
 - 0s - loss: 0.0015
Epoch 26/50
 - 0s - loss: 0.0015
Epoch 27/50
 - 0s - loss: 0.0015
Epoch 28/50
 - 0s - loss: 0.0015
Epoch 29/50
 - 0s - loss: 0.0015
Epoch 30/50
 - 0s - loss: 0.0015
Epoch 31/